In [1]:
import json
import os
from event_model import event_model as em
from validator import validator_lite as vl

# Solvers
from algorithms.track_following import track_following

#
import matplotlib.pyplot as plt
import random
from random import randint
import numpy as np
import pandas as pd
import time

In [2]:
"""
f = open("C:\\Users\\Surface\\Documents\\Uni stuff\\BTR\\Code\\velopix_tracking\\events\\velo_event_0.json")
json_data = json.loads(f.read())
event = em.event(json_data)
print(json_data)
"""

validation_data = []
events = []
for (dirpath, dirnames, filenames) in os.walk("events"):
    for i, filename in enumerate(filenames):
        # Get an event
        f = open(os.path.realpath(os.path.join(dirpath, filename)))
        json_data = json.loads(f.read())
        event = em.event(json_data)
        events.append(event)
        f.close()
        # Append the solution and json_data
        validation_data.append(json_data)
json_data = validation_data

In [3]:
pop_size = 10
fit_size = int(pop_size/2 - 1)

max_pop_score = []
mean_pop_score = []
mean_runtimes = []
best_runtimes = []
best_ind = [0, [(0, 0), (0, 0), 0]]
runtimes = []

In [4]:
def init(size):
    population = []
    for i in range(size):
        max_slopes = random.uniform(0, 1)
        max_tolerance = random.uniform(0, 1)
        max_scatter = random.uniform(0, 1)
        individual = [(max_slopes, max_slopes), (max_tolerance, max_tolerance), max_scatter]
        population.append(individual)
    return population

def evaluate(ind):
    global runtimes
    print("evaluating: " + str(ind))
    (max_slopes, max_slopes), (max_tolerance, max_tolerance), max_scatter = ind
    start_time = time.time()
    agent_tracks = []
    this_agent = track_following((max_slopes, max_slopes), (max_tolerance, max_tolerance), max_scatter)
    for event in events:
        agent_tracks.append(this_agent.solve(event))
    end_time = time.time()

    agent_runtime = end_time - start_time
    runtimes.append(agent_runtime)
    runtime_score = max(1 - agent_runtime/np.mean(runtimes), 0) 

    hit_eff_score = vl.validate_efficiency(json_data, agent_tracks, particle_type='velo')

    clone_score = max(1 - vl.validate_clone_fraction(json_data, agent_tracks, particle_type='velo'), 0)

    ghost_score = max(1 - vl.validate_ghost_fraction(json_data, agent_tracks), 0)

    score = np.mean([hit_eff_score, 
             runtime_score, 
             clone_score, 
             ghost_score])
    
    print("score: " + str(score))
    return score

def get_fit(population):
    global best_ind, runtimes
    scores = []
    # storing the scores of all tracks
    for ind1 in population:
        ind_score = evaluate(ind1)
        scores.append(ind_score)
    av_score = np.mean(scores)
    print("av_score: " + str(av_score))

    fittest = []
    for i in population:
        this_score = evaluate(i)
        if this_score > av_score:
            fittest.append([this_score, i])
            print("added to fittest: " + str(i))
    
    print("n fit before: " + str(len(fittest)))
    fittest.sort(key=lambda tup:tup[0], reverse=False)
    while len(fittest) > fit_size:
        print("pop: " + str(fittest[0]))
        fittest.pop(0)
    
    # uploading best track
    for fit in fittest:
            if fit[1] != best_ind[1] and fit[0] >= best_ind[0]:
                print("NEW best individual: " + str(best_ind))
                best_ind = fit
                best_run_start = time.time()
                (max_slopes, max_slopes), (max_tolerance, max_tolerance), max_scatter = fit[1]
                agent = track_following((max_slopes, max_slopes), (max_tolerance, max_tolerance), max_scatter)
                agent_tracks = agent.solve(event)
                best_run_end = time.time()
                best_rt = best_run_end - best_run_start
                best_runtimes.append(best_rt)
        
    # keeping track of evolution
    max_pop_score.append(max(scores))
    mean_pop_score.append(av_score)
    final_fittest = []
    for r in fittest:
        final_fittest.append(r[1])
    mean_runtimes.append(np.mean(runtimes))
    runtimes = []
    return final_fittest

def mutate(group):
    if len(group) > 0:
        offsprings = []
        print(str(len(group[0])))
        for ind in group:
            pam = randint(0, 2)
            if pam == 0:
                r = random.uniform(0, 1)
                ind = [(r, r), ind[1], ind[2]]
            if pam == 1:
                r = random.uniform(0, 1)
                ind = [ind[0], (r, r), ind[2]]
            if pam == 2:
                ind = [ind[0], ind[1], random.uniform(0, 1)]
            offsprings.append(ind)
    return offsprings


In [5]:
def main(max_gen, population_size, event):
    POPULATION = init(population_size)
    GENERATION = 0
    while GENERATION < max_gen:
        print("gen: " + str(GENERATION))
        fit = get_fit(POPULATION)
        #os = mutate(fit)
        #other = init(population_size - (len(fit) + len(os)))
        other = init(population_size - len(fit))
        print("n fit after: " + str(len(fit)))
        print()
        POPULATION = fit + other
        GENERATION += 1
    print("DONE!")
    



In [6]:
non_op_time_start = time.time()
agent_1 = track_following()
tracks_1 = agent_1.solve(event)
non_op_time_end = time.time()
non_op_runtime = non_op_time_end - non_op_time_start

main(max_gen=100, population_size=pop_size, event=event)

op_time_start = time.time()
best_agent = track_following(best_ind[1][0], best_ind[1][1], best_ind[1][2])
best_tracks = best_agent.solve(event)
op_time_end = time.time()
op_runtime = op_time_end - op_time_start

print()
print("final best: " + str(best_ind))
print(str(len(best_tracks)) + " tracks found")
print()
print(best_tracks)
print()
print("Runtime before: " + str(non_op_runtime))
print("Runtime after: " + str(op_runtime))

Instantiating track_following solver with parameters
 max slopes: (0.7, 0.7)
 max tolerance: (0.4, 0.4)
 max scatter: 0.4

gen: 0
evaluating: [(0.9850450694984669, 0.9850450694984669), (0.07988636754266831, 0.07988636754266831), 0.3318146944628342]
Instantiating track_following solver with parameters
 max slopes: (0.9850450694984669, 0.9850450694984669)
 max tolerance: (0.07988636754266831, 0.07988636754266831)
 max scatter: 0.3318146944628342

score: 0.7094980308789389
evaluating: [(0.5517959269503545, 0.5517959269503545), (0.8588045172268826, 0.8588045172268826), 0.9547830253565679]
Instantiating track_following solver with parameters
 max slopes: (0.5517959269503545, 0.5517959269503545)
 max tolerance: (0.8588045172268826, 0.8588045172268826)
 max scatter: 0.9547830253565679

score: 0.6620256735956369
evaluating: [(0.48632613239917244, 0.48632613239917244), (0.6649256880846666, 0.6649256880846666), 0.07240254017180237]
Instantiating track_following solver with parameters
 max slopes

KeyboardInterrupt: 